In [6]:
%pip uninstall torch torchvision torchaudio -y
%pip cache purge

Found existing installation: torch 2.6.0+cu126
Uninstalling torch-2.6.0+cu126:
  Successfully uninstalled torch-2.6.0+cu126
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.6.0+cu126
Uninstalling torchaudio-2.6.0+cu126:
  Successfully uninstalled torchaudio-2.6.0+cu126
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


Files removed: 1306 (8469.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   - -------------------------------------- 8.1/204.2 MB 46.0 MB/s eta 0:00:05
   --- ------------------------------------ 17.6/204.2 MB 44.4 MB/s eta 0:00:05
   ----- ---------------------------------- 27.0/204.2 MB 43.9 MB/s eta 0:00:05
   ------ --------------------------------- 35.7/204.2 MB 42.7 MB/s eta 0:00:04
   -------- ------------------------------- 44.6/204.2 MB 43.0 MB/s eta 0:00:04
   ---------- ----------------------------- 53.0/204.2 MB 42.2 MB/s eta 0:00:04
   ----------- ---------------------------- 61.1/204.2 MB 41.4 MB/s eta 0:00:04
   ------------- -------------------------- 70.3/204.2 MB 41.9 MB/s eta 0:00:04
   --------------- ------------------------ 79.4/204.2 MB 41.9 MB/s eta 0:00:03
   ----------------- ---------------------- 88.1/204.2 MB 41.6 MB/s eta 0:00:03
   ------------------- -------------------- 97.3/204.2 MB 41.7 M

In [2]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.8,0.1,0.1),group_prefix=None)

Copying files: 2817 files [00:06, 198.89 files/s]

KeyboardInterrupt: 

In [17]:
%pip uninstall torchvision -y


%pip cache purge
%pip install torchvision --index-url https://download.pytorch.org/whl/cu126

Found existing installation: torchvision 0.21.0+cu126
Uninstalling torchvision-0.21.0+cu126:
  Successfully uninstalled torchvision-0.21.0+cu126
Note: you may need to restart the kernel to use updated packages.
Files removed: 4 (6.2 MB)
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 6.1/6.1 MB 34.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

results = model.train(
    results = model.train(
    data="config.yaml",        # Din träningsdata
    project="./output",         # Utmatningsmapp
    device='cuda:0',            # Träna på GPU (ändra om du vill använda CPU)
    epochs=50,                  # Antal epoker
    batch=32,                   # Låt YOLO välja den bästa batch-storleken baserat på GPU-minne
    imgsz=640,                  # Bildstorlek, 640x640 bör vara bra för din GPU
    amp=True,                   # Aktivera Automatic Mixed Precision för snabbare träning och mindre minne
    save_period=5,              # Spara modellen var 5:e epok
    optimizer='adam',           # Använd Adam optimizer för bättre konvergens
    workers=12,                 # Antal arbetare för dataladdning, öka om du har fler CPU-kärnor       
    name="yolo_model",          # Namn på träningssessionen
    patience=10,                # Stoppar om ingen förbättring efter 10 epochs
    exist_ok=True,              # Om du vill skriva över tidigare resultat
)

)
# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
success = model.export(format="onnx")

Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=config.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=5, cache=False, device=cuda:0, workers=12, project=./output, name=yolo_model, exist_ok=True, pretrained=True, optimizer=adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning C:\Projects\VisionComp\cv-25-project\dataset\train\labels.cache... 1763 images, 18 backgrounds, 1 corrupt: 100%|██████████| 1763/1763 [00:00<?, ?it/s]

train: WARNING  C:\Projects\VisionComp\cv-25-project\dataset\train\images\460a9aad-Giant_Slalom_Racer.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Projects\VisionComp\cv-25-project\dataset\train\images\66e2c433-548865909_highres-data-49166682.jpg: corrupt JPEG restored and saved
train: WARNING  C:\Projects\VisionComp\cv-25-project\dataset\train\images\6b2a416b-000030.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1272]
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 3060 Ti) 8.00G total, 0.10G reserved, 0.06G allocated, 7.84G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


     2590230       6.442         0.472         33.77         310.6        (1, 3, 640, 640)                    list
     2590230       12.88         0.633         26.19         115.3        (2, 3, 640, 640)                    list
     2590230       25.77         0.950         25.29         123.8        (4, 3, 640, 640)                    list
     2590230       51.53         1.623         30.98         127.7        (8, 3, 640, 640)                    list
     2590230       103.1         2.892         51.25         133.9       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 38 for CUDA:0 4.83G/8.00G (60%) 


train: Scanning C:\Projects\VisionComp\cv-25-project\dataset\train\labels.cache... 1763 images, 18 backgrounds, 1 corrupt: 100%|██████████| 1763/1763 [00:00<?, ?it/s]

...

      13/50      7.16G      1.468      1.492      1.424         36        640: 100%|██████████| 47/47 [00:23<00:00,  2.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.39it/s]

                   all        220        351      0.688      0.568      0.641      0.343



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/50      7.16G       1.43      1.438      1.426        158        640:   9%|▊         | 4/47 [00:16<02:40,  3.74s/it]

: 